LSTM ile **metin üretimi**

In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [8]:
# GPT ile veri üretimi
texts = [
    "bugün hava çok güzel, dışarıda yürüyüş yapmayı planlıyorum.",
    "kahvemi alıp sakin bir yerde oturmak bana iyi geliyor.",
    "yeni şeyler öğrenmek beni her zaman motive eder.",
    "akşam yemeğinde ev yapımı bir yemek hazırladım.",
    "müzik dinlerken çalışmak daha verimli olmamı sağlıyor.",
    "hafta sonu için küçük bir seyahat planı yapıyorum.",
    "doğada vakit geçirmek zihnimi dinlendiriyor.",
    "uzun zamandır ertelediğim kitabı sonunda okumaya başladım.",
    "spor yaptıktan sonra kendimi daha enerjik hissediyorum.",
    "erken uyanmak günümü daha iyi planlamama yardımcı oluyor.",
    "bugün işler beklediğimden daha hızlı ilerledi.",
    "arkadaşlarımla sohbet etmek moralimi yükseltiyor.",
    "yeni bir diziye başladım ve oldukça sardı.",
    "yoğun bir günün ardından dinlenmeye ihtiyacım var.",
    "hedeflerime adım adım yaklaşmak beni mutlu ediyor.",
    "sessiz bir ortamda çalışmak odaklanmamı artırıyor.",
    "bugün kendime zaman ayırmaya karar verdim.",
    "öğle arasında kısa bir yürüyüş yaptım.",
    "yeni tarifler denemek mutfakta daha yaratıcı olmamı sağlıyor.",
    "yaptığım planlara sadık kalmaya çalışıyorum.",
    "bugün motivasyonum oldukça yüksekti.",
    "teknoloji alanındaki gelişmeleri takip etmeyi seviyorum.",
    "not almak öğrenme sürecimi kolaylaştırıyor.",
    "erken yatmak ertesi gün daha dinç olmamı sağlıyor.",
    "uzun vadeli hedefler belirlemek bana yön veriyor.",
    "bugün biraz yorgun hissediyorum ama üretkenim.",
    "güneşli havalar ruh halimi olumlu etkiliyor.",
    "yeni bir beceri kazanmak için çalışıyorum.",
    "zaman yönetimi konusunda kendimi geliştirmeye çalışıyorum.",
    "sabahları kahve içmek benim için bir rutin.",
    "yoğun tempoya rağmen dengeli kalmaya çalışıyorum.",
    "bugün yapılacaklar listemi tamamladım.",
    "kendime güvenim son zamanlarda arttı.",
    "öğrenirken hata yapmak sürecin bir parçası.",
    "küçük başarılar beni daha çok motive ediyor.",
    "bugün daha pozitif düşünmeye çalıştım.",
    "planlı olmak stresi azaltıyor.",
    "yeni fikirler üretmek beni heyecanlandırıyor.",
    "çalışma ortamımı daha düzenli hale getirdim.",
    "kendimi geliştirmek için kaynaklar araştırıyorum.",
    "bugün verimli bir gün geçirdim.",
    "odaklanınca zamanın nasıl geçtiğini anlamıyorum.",
    "hedeflerime ulaşmak için sabırlı olmam gerekiyor.",
    "gün sonunda yaptıklarımı değerlendirmeyi seviyorum.",
    "düzenli molalar vermek performansımı artırıyor.",
    "bugün biraz daha disiplinliydim.",
    "kendimle ilgili yeni şeyler fark ediyorum.",
    "istikrarlı olmak uzun vadede sonuç veriyor.",
    "bugünü kendim için verimli bir şekilde tamamladım.",
    "bugün yapılacak işlerim beklediğimden fazlaydı.",
    "sabah erken kalkmak başlangıçta zor geldi.",
    "yoğun trafikte zaman kaybetmek moralimi bozdu.",
    "bir süredir ertelediğim işleri tamamlamaya başladım.",
    "uzun bir toplantıdan sonra zihinsel olarak yoruldum.",
    "planladığım gibi gitmeyen şeyler beni düşündürdü.",
    "gün içinde kısa molalar vermek iyi hissettirdi.",
    "beklenmedik bir sorunla karşılaşmak canımı sıktı.",
    "odaklanmakta zorlandığım bir gündü.",
    "akşam eve geç dönmek beni biraz yordu.",
    "bazı konularda daha sabırlı olmam gerektiğini fark ettim.",
    "işleri son ana bırakmak stres seviyemi artırdı.",
    "bugün zaman yönetiminde pek başarılı olamadım.",
    "beklediğim geri dönüşü alamamak hayal kırıklığı yarattı.",
    "yoğunluk nedeniyle kendime vakit ayıramadım.",
    "yaptığım hatalardan ders çıkarmaya çalışıyorum.",
    "gün içinde motivasyonum dalgalıydı.",
    "bazı kararları daha dikkatli almam gerekiyor.",
    "uzun süre ekrana bakmak gözlerimi yordu.",
    "iş yükü arttıkça stresim de arttı.",
    "bugün biraz dağınık hissettim.",
    "plan dışı gelişmeler günü zorlaştırdı.",
    "yorgunluk konsantrasyonumu olumsuz etkiledi.",
    "beklentilerimle sonuçlar örtüşmedi.",
    "bazı görevler tahminimden uzun sürdü.",
    "bugün kendimi pek enerjik hissetmiyorum.",
    "zamanın hızlı geçmesi işleri yetiştirmemi zorlaştırdı.",
    "önceliklendirme yapmam gerektiğini fark ettim.",
    "yoğun tempoda hata yapma ihtimali artıyor.",
    "beklenmedik bir değişiklik planlarımı bozdu.",
    "gün içinde stresle baş etmeye çalıştım.",
    "bazı konular kafamı fazlasıyla meşgul etti.",
    "motivasyonumu yeniden toplamak zor oldu.",
    "bugün biraz isteksizdim.",
    "odak kaybı verimimi düşürdü.",
    "bazı sorumluluklar üst üste geldi.",
    "beklediğimden daha zor bir gündü.",
    "yorgunluk kararlarımı etkiledi.",
    "gün sonunda zihinsel olarak tükendim.",
    "işleri toparlamak zaman aldı.",
    "bugün daha az üretken hissettim.",
    "bazı hedefleri ertelemek zorunda kaldım.",
    "stresli anlarda sakin kalmaya çalıştım.",
    "yoğunluk nedeniyle planlar aksadı.",
    "gün içinde motivasyonumu korumak zordu.",
    "bazı konular üzerinde tekrar düşünmem gerekiyor.",
    "beklentilerimle yüzleştiğim bir gündü.",
    "bugün kendime karşı daha eleştireldim.",
    "yapılacaklar listesi kabardıkça zorlandım.",
    "günü beklediğimden daha yorgun kapattım.",
    "bugünü de böylece tamamlamış oldum."
]

In [9]:
len(texts)

100

In [10]:
# Metin temizleme ve preprocessing
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
total_words = len(tokenizer.word_index) + 1 # toplam kelime sayısı
print(total_words)

369


In [14]:
text_example = texts[0]

sequence = tokenizer.texts_to_sequences([text_example])
print("Sequence:", sequence)

decoded = [tokenizer.index_word[i] for i in sequence[0]]
print("Decoded:", decoded)

Sequence: [[2, 92, 33, 93, 94, 34, 95, 96]]
Decoded: ['bugün', 'hava', 'çok', 'güzel', 'dışarıda', 'yürüyüş', 'yapmayı', 'planlıyorum']


In [21]:
# n-gram dizileri oluştur ve padding uygula
input_sequences = []
for text in texts:
    token_list = tokenizer.texts_to_sequences([text])[0]

    # her metin için n-gram dizisi oluşturalım
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

print(len(input_sequences))
print("\n",input_sequences[0])
print("\n",input_sequences[111])

467

 [2, 92]

 [6, 156, 157, 158, 3, 159, 22]


In [23]:
# en uzun diziyi bulalım ve hepsinin uzunluğunu aynı yapalım : padding
max_sequence_len = max([len(x) for x in input_sequences])

input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

In [24]:
# X (girdi) ve y (hedef değişken)
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

# one hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [25]:
# model

model = Sequential()
# embedding
model.add(Embedding(total_words, 50, input_length=X.shape[1]))

# LSTM
model.add(LSTM(100, return_sequences=False))

# Output
model.add(Dense(total_words, activation='softmax'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [26]:
# compile
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [27]:
# train
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.0122 - loss: 5.9099
Epoch 2/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.0306 - loss: 5.8871
Epoch 3/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.0383 - loss: 5.7547
Epoch 4/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.0521 - loss: 5.5951
Epoch 5/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.0409 - loss: 5.5851
Epoch 6/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.0406 - loss: 5.5880
Epoch 7/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.0283 - loss: 5.5856
Epoch 8/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.0375 - loss: 5.5271
Epoch 9/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.0397 - loss: 5.4753
Epoch 10/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0455 - loss: 5.3596
Epoch 11/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.0448 - loss: 5.2901
Epoch 12/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step

In [28]:
# prediction
def generate_text(seed_text, next_words):
  for _ in range(next_words):

    # girdi metnini sayısal hale getir
    token_list = tokenizer.texts_to_sequences([seed_text])[0]

    # padding
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')

    # prediction
    predicted_probabilities = model.predict(token_list, verbose=0)

    # en yüksek olasılığa sahip kelimenin indexini bul
    predicted_word_index = np.argmax(predicted_probabilities, axis=-1)

    # tokenizer ile kelime indexinden asıl kelimeyi bul
    predicted_word = tokenizer.index_word[predicted_word_index[0]]

    seed_text = seed_text + " " + predicted_word

  return seed_text

In [30]:
seed_text = "Bu hafta sonu"
print(generate_text(seed_text, 6))

Bu hafta sonu için küçük bir seyahat planı yapıyorum
